# Stock Market Predictor (LSTM)

This notebook downloads historical data using `yfinance`, preprocesses it, trains a simple LSTM model using TensorFlow, and visualizes predictions.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Download data
symbol = 'AAPL'
data = yf.download(symbol, start='2018-01-01', end='2024-12-31')
print(data.tail())

In [ ]:
# Preprocess
scaler = MinMaxScaler()
prices = data['Close'].values.reshape(-1,1)
prices_scaled = scaler.fit_transform(prices)

# create sequences
seq_len = 60
X, y = [], []
for i in range(seq_len, len(prices_scaled)):
    X.append(prices_scaled[i-seq_len:i, 0])
    y.append(prices_scaled[i, 0])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

# train/test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    LSTM(32),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=5, batch_size=32)

# Predict
preds = model.predict(X_test)
preds_inv = scaler.inverse_transform(preds)
actual_inv = scaler.inverse_transform(y_test.reshape(-1,1))

# Plot
plt.figure(figsize=(10,6))
plt.plot(actual_inv, label='Actual')
plt.plot(preds_inv, label='Predicted')
plt.legend()
plt.title('Stock Price Prediction - Test Set')
plt.show()